## 1.1 Contents<a id='1.1_Contents'></a>
* [1 Data Introduction](#1_Data_intorduction)
  * [1.1 Objectives](#1.4_Objectives)
    * [1.1.1 Key Questions](#1.3_Imports)
  * [1.2 Product Dataset](#1.1_Product_Information)
  * [1.3 Sales History Dataset](#1.2_Sales_History)
  * [1.4 Load The Ski Resort Data](#1.5_Load_The_Ski_Resort_Data)
  * [1.5 Explore The Data](#1.6_Explore_The_Data)
    * [1.5.1 Find Your Resort Of Interest](#1.6.1_Find_Your_Resort_Of_Interest)
    * [1.5.2 Number Of Missing Values By Column](#1.6.2_Number_Of_Missing_Values_By_Column)
    * [1.5.3 Categorical Features](#1.6.3_Categorical_Features)
      * [1.5.3.1 Unique Resort Names](#1.6.3.1_Unique_Resort_Names)
      * [1.6.3.2 Region And State](#1.6.3.2_Region_And_State)
      * [1.6.3.3 Number of distinct regions and states](#1.6.3.3_Number_of_distinct_regions_and_states)
      * [1.6.3.4 Distribution Of Resorts By Region And State](#1.6.3.4_Distribution_Of_Resorts_By_Region_And_State)
      * [1.6.3.5 Distribution Of Ticket Price By State](#1.6.3.5_Distribution_Of_Ticket_Price_By_State)
 * [1.7 Summary](#1.7_Summary)

# 1. Data Introduction<a id='1_Data_Introduction'></a>

Sunlight Ski and Bike, a small store in Glenwood Springs, relies on sales representatives' suggestions for seasonal orders, leading to inefficient inventory management and buying due to:
* Lack of understanding of product inventory and demand.
* Unclear product categories: prioritizing or eliminating categories.
* Difficulty identifying high-demand, high-value items.
* Missed opportunities for cross-selling.

This results in:
* Stockouts for in-demand products.
* Overstocking on slow-moving items.
* Lost revenue and customer satisfaction.

## Objectives<a id='1.1_Objectives'></a>

This project will focus on examining two data sources: 
(1) Historical sales data (product ID, SKU, name, price, quantity sold, date, time, payment method)
(2) Product inventory data (current stock levels, product descriptions, features, specifications)

in order to gain an understanding of how to <b>optimize product inventory and sales to maximize profitability and customer satisfaction</b>.

Specific questions:
* Which products sell the best and the quickest?
* What factors influence sales patterns (e.g., seasonality, promotions, external events)?
* Can we accurately predict future demand for specific products or categories?
* How can we personalize product recommendations to individual customers?

In [1]:
#import libraries
import os
import warnings
warnings.filterwarnings("ignore")
import numpy as np 
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# sales_csv = glob.glob('c:/Users/{username}/Documents/Data Science Projects/sunlight-retail-analysis/data/raw/sales/*.csv')

# sales_frame = []
# for file in sales_csv:
#     sales_frame.append(pd.read_csv(sales_csv))
                 
# sales = pd.concat(sales_frame)                 

sales_csv = "../data/raw/Sales 1-3-2024.csv"
sales_category_excel = "Sales_By_Category_20231226_1648.xlsx"
sales_history_csv = "../data/raw/2023 - Sunlight Ski and Bike Shop.csv"

## 1. 1 Product Datasets<a id='#1.1_Product_Information'></a>

Let's look at the products dataset and clean up the category information to get a better understanding of the types of products.

In [2]:
# Load datasets
product_excel = "../data/raw/In-Store_Products_202401021512.xlsx" 

# Read the Excel file into a DataFrame
products = pd.read_excel(product_excel)

# Optionally, view the first few rows of the DataFrame
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121355 entries, 0 to 121354
Data columns (total 31 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ID                  121355 non-null  object 
 1   SKU                 121355 non-null  object 
 2   UPC                 118752 non-null  object 
 3   MPN                 19424 non-null   object 
 4   VPN                 116916 non-null  object 
 5   Category            21396 non-null   object 
 6   Description         121355 non-null  object 
 7   Keyword             120677 non-null  object 
 8   EstimatedCost       121355 non-null  float64
 9   MSRP                121355 non-null  float64
 10  Quantity            121355 non-null  int64  
 11  Min                 121355 non-null  int64  
 12  Min2                121355 non-null  int64  
 13  Max                 121355 non-null  int64  
 14  Max2                121355 non-null  int64  
 15  IsNonInventory      121355 non-nul

In [3]:
# products.dropna(subset=['Category'])
products['Category'] = products['Category'].astype(str)
products['ParentCategory'] = products['Category'].apply(lambda x: x.split('-')[0])

In [24]:
# products = products.set_axis(['Category', *products.columns[1:]], axis=1, inplace=False)
products = products.reset_index(drop=True)
parent_categories = products.iloc[:, 31].tolist()
print(len(parent_categories))


121355


In [ ]:
clean_products = products[["Category", "Description","Keyword","UPC","MSRP","Quantity","SKU","Color","Size",
"StyleNumber", "ParentCategory"]]

In [ ]:
categories = clean_products["Category"].unique
clean_products.head()


It appears that there are a roughly 121,355 entries of the vendor products. This does not show what we have in inventory. This is a complete list of all of the product that we stock. And it looks like there are no duplicated products: 

In [ ]:
clean_products['Category'] = clean_products['ParentCategory'].astype(str)
clean_products.info(verbose = False)
clean_products.duplicated().sum()

In [ ]:
clean_products.groupby('Category').sum()


In [ ]:
clean_products['Category'] = clean_products['Category'].astype('category')
print(clean_products['Category'].dtype)

In [ ]:
clean_products.info()
clean_products['Description'] = clean_products['Description'].astype('string')
clean_products['Keyword'] = clean_products['Keyword'].astype('string')
clean_products['UPC'] = clean_products['UPC'].astype('string')
clean_products['Color'] = clean_products['Color'].astype('string')
clean_products['Size'] = clean_products['Size'].astype('category')

In [ ]:
#lets look at the category column as a way to get insight on the products
unique_counts = clean_products['Category'].unique()
print(unique_counts)
# clean_products['Category'].nunique()

print(clean_products['Description'].nunique())

In [ ]:
sns.countplot(x='ParentCategory',data=clean_products,palette='tab10')
plt.xticks(rotation=45)

In [ ]:
nan_rows = clean_products[pd.isna(clean_products['ParentCategory'])]
print(nan_rows)

## 1. 2 Sales History Dataset<a id='#1.2_Sales_History'></a>

In [ ]:
# sales = pd.read_csv(sales_csv, encoding='latin-1')
sales = pd.read_csv(sales_history_csv, encoding='latin-1')
print(sales.head())
print(sales.info())

In [ ]:
sales.describe()

In [ ]:
sales_category = pd.read_excel(sales_category_excel)
print(sales_category.head())

In [ ]:
sorted_categories = unique_counts.sort()

# 1.7 Summary<a id='1.7_Summary'></a>

There are a number of missing category values for the products, which makes it hard to run reports. For now, I have dropped the ;nan' values. 